<a href="https://colab.research.google.com/github/OlenaPolyvoda/Data-analysis-projects/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
import pandas as pd

# Визначення метрик
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new_account']

def build_total_table(df, metrics):
    results = []
    for test_id in df['test'].unique():
        test_df = df[df['test'] == test_id]
    for metric in metrics:
        filtered = df[df['event_name'].isin([metric, 'session'])]
        grouped = filtered.groupby(['test_group', 'event_name'])['value'].sum().unstack(fill_value=0)

        if set([1, 2]).issubset(grouped.index):
            control = grouped.loc[1]
            test = grouped.loc[2]

            conv_control = control[metric] / control['session']
            conv_test = test[metric] / test['session']
            change = (conv_test - conv_control) / conv_control * 100

            stat, pval = proportions_ztest([test[metric], control[metric]],
                                           [test['session'], control['session']])

            results.append({
                'test_id': test_id,
                'metric': metric,
                'numerator_control': int(control[metric]),
                'denominator_control': int(control['session']),
                'conversion_rate_control': conv_control,
                'numerator_test': int(test[metric]),
                'denominator_test': int(test['session']),
                'conversion_rate_test': conv_test,
                'metric_change_%': change,
                'z_stat': stat,
                'p_value': pval,
                'significant': pval < 0.05})
    return pd.DataFrame(results)

# Виклик функції
df = pd.read_csv("/портфоліо 2.csv")
result_df = build_total_table(df, metrics)
print(result_df)

   test_id             metric  numerator_control  denominator_control  \
0        3   add_payment_info              11686               271125   
1        3  add_shipping_info              16940               271125   
2        3     begin_checkout              30133               271125   
3        3        new_account              22828               271125   

   conversion_rate_control  numerator_test  denominator_test  \
0                 0.043102           11936            271017   
1                 0.062480           16875            271017   
2                 0.111141           29865            271017   
3                 0.084197           22374            271017   

   conversion_rate_test  metric_change_%    z_stat   p_value  significant  
0              0.044042         2.180014  1.694549  0.090161        False  
1              0.062265        -0.344010 -0.327211  0.743508        False  
2              0.110196        -0.849895 -1.108462  0.267662        False  
3        

In [ ]:
from google.colab import files
result_df.to_csv("metrics_total.csv", index=False)
df.to_csv('output.csv', index=False, encoding='utf-8')
files.download("metrics_total.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Статистично значущою є тільки одна метрика — new_account (p = 0.029), що менше 0,05.

Кореляція між контрольними та тестовими конверсіями дуже сильна (метрики стабільні).

Зміни в метриках мають сильну лінійну залежність від z-статистики.

[статистичний дешборд](https://public.tableau.com/app/profile/olena.polivoda/viz/statisticalsignificance/statsignificance?publish=yes)